<a href="https://colab.research.google.com/github/oliverguhr/deep-nlp-workshop/blob/main/workshop_transformer_offensive_language_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Offensive Language Classification


## Vorbereitung

Zurerst laden wir uns die notwendigen Daten und Pakete herrunter.

In [ ]:
!pip install datasets transformers

In [ ]:
!wget -c https://www.htw-dresden.de/~guhr/dist/sample/germeval2018.training.txt
!wget -c https://www.htw-dresden.de/~guhr/dist/sample/germeval2018.test.txt

In [ ]:
import time
import pandas as pd
import numpy as np

## Vorbereiten der Daten

Im nächsten Schritt müssen wir die Daten laden und etwas anpassen. Die Daten liegen Tabstopp getrennte csv vor. Für die einfache Verarbeitung bietet sich Pandas an, könnte aber auch mit Python boardmitteln erledigt werden.

In [ ]:
test_df = pd.read_csv("germeval2018.test.txt", sep='\t', header=0,encoding="utf-8")
train_df = pd.read_csv("germeval2018.training.txt", sep='\t', header=0,encoding="utf-8")

In [ ]:
train_df.head()

In [ ]:
# Die nicht benötigten Spalten werden gelöscht
test_df.drop(columns=['label2'], inplace=True)
train_df.drop(columns=['label2'], inplace=True)

In [ ]:
def clean_text (text):
    #text = text.str.lower() # lowercase
    #text = text.str.replace(r"\#","") # replaces hashtags
    #text = text.str.replace(r"http\S+","URL")  # remove URL addresses
    #text = text.str.replace(r"@","")
    #text = text.str.replace(r"[^A-Za-z0-9öäüÖÄÜß()!?]", " ")
    #text = text.str.replace("\s{2,}", " ")
    return text

def convert_label(label):
    return 1 if label == "OFFENSE" else 0

In [ ]:
train_df["text"]=clean_text(train_df["text"])
test_df["text"]=clean_text(test_df["text"])
train_df["label"]=train_df["label"].map(convert_label)
test_df["label"]=test_df["label"].map(convert_label)

In [ ]:
# und jetzt sehen unsere Daten so aus
train_df.head() 

In [ ]:
len(train_df.loc[train_df["label"]==1])

Wieviele Datensätze haben wir in unserem Train/Valid/Test sets?

In [ ]:
print(f"Test exampels \t {len(test_df) }")
print(f"Train exampels \t {len(train_df[500:])}")
print(f"Valid exampels \t {len(train_df[:500])}")

In [ ]:
# Im letzten schritt convertieren wir unsere Daten in ein Format, welches die ML Bibliothek nutzen kann.

from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df[500:])
valid_dataset = Dataset.from_pandas(train_df[:500])
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# Wie sieht unser dataset jetzt aus?
train_dataset

## Encoding der Daten

Wir wandeln unsere Texte mit dem Tokenizer des Models in Vekoren um, die unser Model verarbeiten kann. 

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric


# was passiert wenn wir "distilbert-base-multilingual-cased" nutzen?

model_checkpoint ="distilbert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
demo_tokens = tokenizer(["Mehr Daten führen oftmals zu besseren Ergebnissen.", "And this is a second sentence"], truncation=True)
demo_tokens

In [ ]:
tokenizer.convert_ids_to_tokens(demo_tokens['input_ids'][0])

In [ ]:
def data_tokenizer(examples):
    return tokenizer(examples["text"], truncation=True,padding=False)

In [ ]:
encoded_train_dataset = train_dataset.map(data_tokenizer, batched=True)
encoded_valid_dataset = valid_dataset.map(data_tokenizer, batched=True)
encoded_test_dataset = test_dataset.map(data_tokenizer, batched=True)

In [ ]:
encoded_train_dataset

## Das Training \o/

Nun können wir unser Model trainieren. Dazu müssen wir noch eine Reihe von Einstellungen (Hyperparameter) festlegen:

In [ ]:
metric = load_metric('f1')

def compute_metrics(eval_pred):    
    predictions, labels = eval_pred    
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

batch_size = 16

args = TrainingArguments(
    "test-offsive-language",
    evaluation_strategy = "epoch",
    save_strategy ="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    warmup_steps=100,
    logging_steps=10,
    weight_decay=0.001,
    load_best_model_at_end=True,
    metric_for_best_model="f1", 
    fp16=True   
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_valid_dataset,    
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
# Wieviel GPU Speicher haben wir belegt? 
!nvidia-smi
#import torch
#torch.cuda.empty_cache()
#!nvidia-smi

In [ ]:
#tensorboard --logdir runs
#%load_ext tensorboard
#%reload_ext tensorboard
%tensorboard --logdir ./test-offsive-language/runs

## Tests

Im nächsten Schritt können wir testen wir unser Modell mit den bereitgestellten Testdaten.

In [ ]:
result = trainer.predict(encoded_test_dataset)
result.metrics["test_f1"]

In [ ]:
import torch

#trainer.prediction_step(trainer.model,tokenizer("das ist ein test"),False)
trainer.model.cpu()
#trainer.model.num_parameters()
encoded_texts = tokenizer(["du bist so dumm", "du bist toll"],padding=True, return_tensors="pt")
print(encoded_texts)
logits = trainer.model(**encoded_texts)
probabilities = torch.softmax(logits[0],dim=1)
print(probabilities)
class_label = torch.argmax(probabilities,dim=1)
print(class_label)

In [ ]:
# Wie lange dauert es eine einzelne Vorhersage zu berechnen?

def predict(text):
    trainer.model.cpu()
    #trainer.model.num_parameters()
    encoded_texts = tokenizer(text, return_tensors="pt")
    #print(encoded_texts)
    logits = trainer.model(**encoded_texts)
    probabilities = torch.softmax(logits[0],dim=1)
    #print(probabilities)
    class_label = torch.argmax(probabilities)
    return class_label
    #print(class_label)

%timeit predict("du bist so toll")



# Jetzt bist du drann :)

Unsere Ergebnisse sind schon ganz gut - aber wir können die Ergebisse noch verbessern. 
Mache dich zuerst mit dem Notebook vertraut - ändere ein paar Parameter wie Lernrate und die Anzahl der Epochen und schau dir an wie diese das Ergebiss verändern. 

Hier sind bei paar Ideen für deine Experimente:

* Wir haben das Modell erst eine Epoch trainiert. Was passtiert wenn wir es zwei oder drei Epochen trainieren?

* Teste verschiedene Modelle aus. Der [Model Hub](https://huggingface.co/models) listet eine Reihe von deutschen modellen mit denen du die Ergebnisse verbessern kannst. 

* Rund 5000 Datensätze sind vergleichweise wenig für dieses Problem. Evtl. findest du weitere Datensätze die du zum aktuellen Trainingsdatensatz hinzufügen kannst.

* Im [Model Hub](https://huggingface.co/models) stehen eine Reihe von multilingualen Modellen zur Verfügung. Diese Modelle wurden mit verschiedenen Sprachen trainiert. Du könntest ebenfalls versuchen den deutschen Datensatz einen englischen hinzuzufügen um ein mehrsprachiges Modell zu trainieren. Möglicherweise ist dieses auch auf den deutschen Daten besser. 

Data Augmentation ist ein Verfahren um neue Datensätze zu erzeugen, in dem man bestehende Datensätze etwas modifiziert. Wichtig ist dabei, das sich die Aussage nicht ändert (die Klasse die gleiche bleibt)

* Du kannst Synonyme Wörter ersetzen und so neue Datensätze generieren. Ein Beispiel:

> "Kann man diesen ganzen Scheiß noch glauben?" -> "Kann man diesen ganzen Mist noch glauben?"

* Alles ist hier erlaubt. Versuche Texte von Deutsch nach Englisch und wieder nach Deutsch zu übersetzen. Wenn der Sinn erhalten kann das Ergebniss auch für das Training genutzt werden. Ein kleines Beispiel mit Google Translate:

> Deutsch: "Kann man diesen ganzen Scheiß noch glauben?" 

> Englisch: "Can you still believe all this shit?"

> Deutsch: "Kannst du all diese Scheiße noch glauben?"


